In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv(
    "../data/sentiment.csv",
    encoding="latin-1"
)

df.head()


,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [2]:
df.info()
df['sentiment'].value_counts()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   textID            27481 non-null  object 
 1   text              27480 non-null  object 
 2   selected_text     27480 non-null  object 
 3   sentiment         27481 non-null  object 
 4   Time of Tweet     27481 non-null  object 
 5   Age of User       27481 non-null  object 
 6   Country           27481 non-null  object 
 7   Population -2020  27481 non-null  int64  
 8   Land Area (Km²)   27481 non-null  float64
 9   Density (P/Km²)   27481 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 2.1+ MB


sentiment
neutral     11118
positive     8582
negative     7781
Name: count, dtype: int64

In [3]:
df['sentiment'] = df['sentiment'].map({
    'negative': 0,
    'neutral': 1,
    'positive': 2
})

df['sentiment'].value_counts()


sentiment
1    11118
2     8582
0     7781
Name: count, dtype: int64

In [ ]:
# import re
# import nltk
# from nltk.corpus import stopwords

# nltk.download('stopwords')

# def clean_text(text):
#     text = str(text).lower()
#     text = re.sub(r'[^a-zA-Z]', ' ', text)
#     text = text.split()
#     text = [w for w in text if w not in stopwords.words('english')]
#     return ' '.join(text)

# df['clean_text'] = df['text'].apply(clean_text)
# df[['text', 'clean_text']].head()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,clean_text
0,"I`d have responded, if I were going",responded going
1,Sooo SAD I will miss you here in San Diego!!!,sooo sad miss san diego
2,my boss is bullying me...,boss bullying
3,what interview! leave me alone,interview leave alone
4,"Sons of ****, why couldn`t they put them on t...",sons put releases already bought


In [18]:
import re

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'[^a-zA-Z ]', '', text)
    return text

df['clean_text'] = df['text'].apply(clean_text)


In [19]:
from sklearn.model_selection import train_test_split

X = df['clean_text']
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf = TfidfVectorizer( max_features=8000,
#     ngram_range=(1,2))

# X_train_tfidf = tfidf.fit_transform(X_train)
# X_test_tfidf = tfidf.transform(X_test)


In [20]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_WORDS = 10000
MAX_LEN = 150

tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post')


In [8]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, classification_report

# model = LogisticRegression(max_iter=1000,class_weight="balanced")
# model.fit(X_train_tfidf, y_train)

# y_pred = model.predict(X_test_tfidf)

# print("Accuracy:", accuracy_score(y_test, y_pred))
# print(classification_report(y_test, y_pred))


In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

model = Sequential([
    Embedding(MAX_WORDS, 128, input_length=MAX_LEN),
    Bidirectional(LSTM(64)),
    Dropout(0.5),
    Dense(64, activation="relu"),
    Dropout(0.3),
    Dense(3, activation="softmax")
])

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)
model.summary()

e:\Sentimental_Analysis\venv\lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [22]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train
)

class_weight_dict = dict(enumerate(class_weights))


In [23]:
history = model.fit(
    X_train_pad,
    y_train,
    epochs=15,
    batch_size=64,
    validation_split=0.2,
    class_weight=class_weight_dict
)


Epoch 1/15
275/275 ━━━━━━━━━━━━━━━━━━━━ 38s 127ms/step - accuracy: 0.5416 - loss: 0.8978 - val_accuracy: 0.6466 - val_loss: 0.7867
Epoch 2/15
275/275 ━━━━━━━━━━━━━━━━━━━━ 34s 123ms/step - accuracy: 0.7310 - loss: 0.6301 - val_accuracy: 0.6916 - val_loss: 0.7168
Epoch 3/15
275/275 ━━━━━━━━━━━━━━━━━━━━ 33s 121ms/step - accuracy: 0.7977 - loss: 0.5011 - val_accuracy: 0.6766 - val_loss: 0.7582
Epoch 4/15
275/275 ━━━━━━━━━━━━━━━━━━━━ 34s 122ms/step - accuracy: 0.8331 - loss: 0.4155 - val_accuracy: 0.6761 - val_loss: 0.8397
Epoch 5/15
275/275 ━━━━━━━━━━━━━━━━━━━━ 34s 123ms/step - accuracy: 0.8633 - loss: 0.3514 - val_accuracy: 0.6682 - val_loss: 0.9101
Epoch 6/15
275/275 ━━━━━━━━━━━━━━━━━━━━ 34s 124ms/step - accuracy: 0.8846 - loss: 0.2996 - val_accuracy: 0.6732 - val_loss: 1.0208
Epoch 7/15
275/275 ━━━━━━━━━━━━━━━━━━━━ 34s 123ms/step - accuracy: 0.9038 - loss: 0.2520 - val_accuracy: 0.6648 - val_loss: 1.1535
Epoch 8/15
275/275 ━━━━━━━━━━━━━━━━━━━━ 33s 121ms/step - accuracy: 0.9176 - loss: 0

In [24]:
loss, accuracy = model.evaluate(X_test_pad, y_test)
print("Test Accuracy:", accuracy)


172/172 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.6505 - loss: 2.0452
Test Accuracy: 0.6505366563796997


In [ ]:
# import pickle

# with open("../src/sentiment_model.pkl", "wb") as f:
#     pickle.dump(model, f)

# with open("../src/tfidf_vectorizer.pkl", "wb") as f:
#     pickle.dump(tfidf, f)

# print("Model saved successfully!")


Model saved successfully!


In [25]:
import pickle

model.save("../src/sentiment_dl_model.h5")

with open("../src/tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

print("DL model and tokenizer saved successfully!")


DL model and tokenizer saved successfully!
